# MIRCF — Full Notebook: Classical Multi-Agent + Quantum Integration

This notebook contains an interactive dashboard that:

- Runs the classical MIRCF multi-agent simulation (RSI, EDB, RM, expanders)
- Visualizes agent trajectories, RSI heatmaps, entropy heatmaps, spectral-gap diagnostics
- Provides an integrated quantum density-matrix simulation for a selected agent linked to its classical trajectory

**Requirements:** `ipywidgets` for interactivity. `networkx` is optional (nicer adjacency graphs).

**How to use:** Run the large code cell below. First press **Run simulation** (classical) then choose **Quantum agent** and press **Run quantum sim for selected agent**.



In [ ]:

# MIRCF: Multi-Agent + Quantum Integrated Dashboard (single cell)
# Paste this cell into a Jupyter notebook and run. Requires ipywidgets; networkx optional.
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import (
    VBox, HBox, FloatSlider, IntSlider, Dropdown, Button, Checkbox, Output, Label, SelectMultiple, ToggleButtons
)
import math, warnings, sys
from IPython.display import display, clear_output

# Optional dependency networkx for nicer adjacency drawing
try:
    import networkx as nx
    HAS_NX = True
except Exception:
    HAS_NX = False

# -------------------------
# MIRCF classical primitives
# -------------------------
def abel_regularize(x, eps):
    return x / (1 + eps * np.abs(x))

def EDB_scalar(x, sigma=0.1):
    p = 0.5 * (1 + np.tanh(x / sigma))
    p = np.clip(p, 1e-9, 1-1e-9)
    return -p * np.log2(p) - (1-p) * np.log2(1-p)

def spectral_gap(adj):
    vals = np.linalg.eigvals(adj)
    vals = np.sort(np.real(vals))[::-1]
    if vals.size < 2:
        return 0.0
    return float(vals[0] - vals[1])

def generate_adj(N, mode='ring', p=0.1, d=3, seed=None):
    rng = np.random.default_rng(seed)
    if mode == 'ring':
        A = np.zeros((N,N), dtype=float)
        for i in range(N):
            A[i,(i+1)%N] = 1
            A[i,(i-1)%N] = 1
        return A
    elif mode == 'fully_connected':
        A = np.ones((N,N), dtype=float) - np.eye(N)
        return A
    elif mode == 'erdos_renyi':
        A = (rng.random((N,N)) < p).astype(float)
        np.fill_diagonal(A, 0)
        A = np.triu(A,1)
        A = A + A.T
        return A
    elif mode == 'random_regular':
        if HAS_NX:
            try:
                G = nx.random_regular_graph(d, N, seed=seed)
                A = nx.to_numpy_array(G, dtype=float)
                return A
            except Exception:
                warnings.warn("networkx.random_regular_graph failed; falling back")
        A = np.zeros((N,N), dtype=float)
        k = d
        for i in range(N):
            for j in range(1, k//2 + 1):
                A[i,(i+j)%N] = 1
                A[i,(i-j)%N] = 1
        np.fill_diagonal(A, 0)
        return A
    elif mode == 'ramanujan_bipartite':
        N_total = N
        N1 = N_total // 2
        N2 = N_total - N1
        d_left = min(max(1, d), N2)
        rng = np.random.default_rng(seed)
        A = np.zeros((N_total,N_total), dtype=float)
        for i in range(N1):
            neighbors = rng.choice(np.arange(N1, N1+N2), size=d_left, replace=False)
            for nb in neighbors:
                A[i, nb] = 1
                A[nb, i] = 1
        # small rewiring
        rewiring_prob = 0.05
        for i in range(N1):
            for j in range(N1, N1+N2):
                if rng.random() < rewiring_prob:
                    A[i,j] = 1 - A[i,j]
                    A[j,i] = A[i,j]
        np.fill_diagonal(A, 0)
        return A
    else:
        raise ValueError("Unknown adjacency mode: "+str(mode))

def row_normalize(A):
    row_sums = A.sum(axis=1, keepdims=True)
    row_sums[row_sums==0] = 1.0
    return A / row_sums

def mircf_update_vector(x_t, x_t_minus1, a, b, eps, K_RSI=0.6, K_EDB=0.8, gamma=0.2, W=None, lam=0.1, sigma=0.1):
    x_raw = a * x_t + b * x_t_minus1
    x_reg = abel_regularize(x_raw, eps)
    rsi = np.exp(-lam * np.abs(x_reg - x_t))
    edb = np.array([EDB_scalar(v, sigma) for v in x_reg])
    rho_min = 0.5
    u_RSI = -K_RSI * np.maximum(0.0, rho_min - rsi) * x_reg
    u_EDB = -K_EDB * edb * np.sign(x_reg)
    coupling = np.zeros_like(x_t)
    if W is not None:
        coupling = gamma * (W @ x_t - x_t)
    x_next = x_reg + u_RSI + u_EDB + coupling
    diagnostics = {'raw': x_raw, 'reg': x_reg, 'rsi': rsi, 'edb': edb, 'u_RSI': u_RSI, 'u_EDB': u_EDB, 'coupling': coupling}
    return x_next, diagnostics

# -------------------------
# Simple quantum primitives (2x2 density matrices)
# -------------------------
def bloch_to_rho(b_vec):
    # b_vec: (x,y,z) with norm <= 1
    x,y,z = b_vec
    I = np.eye(2, dtype=complex)
    sx = np.array([[0,1],[1,0]], dtype=complex)
    sy = np.array([[0,-1j],[1j,0]], dtype=complex)
    sz = np.array([[1,0],[0,-1]], dtype=complex)
    rho = 0.5 * (I + x*sx + y*sy + z*sz)
    # numerical cleanup
    rho = (rho + rho.conj().T) / 2
    rho = rho / np.trace(rho)
    return rho

def rho_to_bloch(rho):
    # return (x,y,z)
    sx = np.array([[0,1],[1,0]], dtype=complex)
    sy = np.array([[0,-1j],[1j,0]], dtype=complex)
    sz = np.array([[1,0],[0,-1]], dtype=complex)
    x = np.real(np.trace(rho @ sx))
    y = np.real(np.trace(rho @ sy))
    z = np.real(np.trace(rho @ sz))
    return np.array([x,y,z])

def purity(rho):
    return np.real(np.trace(rho @ rho))

def von_neumann_entropy(rho):
    vals = np.linalg.eigvals(rho)
    vals = np.real(vals)
    vals = np.clip(vals, 1e-12, 1.0)
    return -np.sum(vals * np.log2(vals))

def dephasing_channel(rho, p):
    # Kraus: sqrt(1-p) I, sqrt(p) Z
    K0 = np.sqrt(1-p) * np.eye(2, dtype=complex)
    K1 = np.sqrt(p) * np.array([[1,0],[0,-1]], dtype=complex)
    out = K0 @ rho @ K0.conj().T + K1 @ rho @ K1.conj().T
    return out

def depolarizing_channel(rho, p):
    I = np.eye(2, dtype=complex)
    return (1-p)*rho + p*(I/2)

def unitary_rotation(rho, theta, axis='y'):
    if axis=='y':
        U = np.array([[np.cos(theta/2), -np.sin(theta/2)],[np.sin(theta/2), np.cos(theta/2)]], dtype=complex)
    elif axis=='x':
        U = np.array([[np.cos(theta/2), -1j*np.sin(theta/2)],[-1j*np.sin(theta/2), np.cos(theta/2)]], dtype=complex)
    else:
        U = np.array([[np.exp(-1j*theta/2),0],[0, np.exp(1j*theta/2)]], dtype=complex)
    return U @ rho @ U.conj().T

# -------------------------
# Dashboard widgets
# -------------------------
N_slider     = IntSlider(value=24, min=4, max=300, step=1, description='N')
T_slider     = IntSlider(value=300, min=50, max=1000, step=10, description='T')
mode_drop    = Dropdown(options=['ring','erdos_renyi','random_regular','fully_connected','ramanujan_bipartite'],
                        value='ramanujan_bipartite', description='graph')
p_slider     = FloatSlider(value=0.12, min=0.01, max=0.5, step=0.01, description='p (ER)')
d_slider     = IntSlider(value=4, min=2, max=40, step=1, description='degree d')
a_slider     = FloatSlider(value=1.02, min=0.5, max=1.5, step=0.01, description='a')
b_slider     = FloatSlider(value=0.98, min=0.0, max=3.0, step=0.01, description='b')
eps_slider   = FloatSlider(value=0.02, min=0.0, max=0.5, step=0.002, description='eps')
gamma_slider = FloatSlider(value=0.15, min=0.0, max=1.0, step=0.01, description='gamma')
K_RSI_slider = FloatSlider(value=0.6, min=0.0, max=2.0, step=0.05, description='K_RSI')
K_EDB_slider = FloatSlider(value=0.8, min=0.0, max=2.0, step=0.05, description='K_EDB')
agent_select  = SelectMultiple(options=list(range(0,24)), value=(0,1,2), description='Highlight')
quantum_agent  = Dropdown(options=list(range(0,24)), value=0, description='Quantum agent')
run_button   = Button(description='Run simulation', button_style='primary')
run_quantum_button = Button(description='Run quantum sim for selected agent', button_style='info')
plot_adj_chk = Checkbox(value=True, description='Plot adjacency')
out_area     = Output(layout={'border': '1px solid gray', 'max_height': '800px', 'overflow': 'auto'})
status_lbl   = Label(value='Ready')
quantum_status = Label(value='Quantum ready')

controls_row1 = HBox([N_slider, T_slider, mode_drop, d_slider, p_slider])
controls_row2 = HBox([a_slider, b_slider, eps_slider, gamma_slider])
controls_row3 = HBox([K_RSI_slider, K_EDB_slider, plot_adj_chk, run_button, status_lbl])
controls_row4 = HBox([Label(value='Highlight agents (select multiple):'), agent_select, Label(value='Quantum agent:'), quantum_agent, run_quantum_button, quantum_status])

dashboard = VBox([new_label:=Label(value="MIRCF Multi-Agent + Quantum Dashboard"), controls_row1, controls_row2, controls_row3, controls_row4, out_area])
display(dashboard)

# -------------------------
# Simulation runner (classical)
# -------------------------
def run_multi_agent(_):
    out_area.clear_output(wait=True)
    status_lbl.value = 'Running...'
    N = int(N_slider.value)
    T = int(T_slider.value)
    mode = mode_drop.value
    p = float(p_slider.value)
    d = int(d_slider.value)
    a = float(a_slider.value)
    b = float(b_slider.value)
    eps = float(eps_slider.value)
    gamma = float(gamma_slider.value)
    K_RSI = float(K_RSI_slider.value)
    K_EDB = float(K_EDB_slider.value)

    A = generate_adj(N, mode=mode, p=p, d=d, seed=42)
    W = row_normalize(A)

    rng = np.random.default_rng(0)
    x0 = rng.normal(loc=0.0, scale=0.6, size=N)
    x_prev = rng.normal(loc=0.0, scale=0.6, size=N)

    X = np.zeros((T, N))
    RSI_ts = np.zeros((T, N))
    EDB_ts = np.zeros((T, N))
    V_ts = np.zeros(T)
    X[0,:] = x_prev
    X[1,:] = x0

    for t in range(1, T-1):
        x_next, diag = mircf_update_vector(
            X[t,:], X[t-1,:], a, b, eps,
            K_RSI=K_RSI, K_EDB=K_EDB, gamma=gamma, W=W
        )
        X[t+1,:] = x_next
        RSI_ts[t+1,:] = diag['rsi']
        EDB_ts[t+1,:] = diag['edb']
        mean_vec = X[t+1,:].mean()
        V_ts[t+1] = 0.5 * np.sum((X[t+1,:] - mean_vec)**2) + np.sum(np.maximum(0.0, 0.5 - RSI_ts[t+1,:])**2)

    gap = spectral_gap(A)

    selected = list(agent_select.value)
    if len(selected) == 0:
        selected = list(range(min(6, N)))

    with out_area:
        fig = plt.figure(figsize=(16,12))
        gs = fig.add_gridspec(3,4, height_ratios=[1,1,1.2])
        ax_states = fig.add_subplot(gs[0, :2])
        ax_rsi_heat = fig.add_subplot(gs[0, 2:])
        ax_entropy_heat = fig.add_subplot(gs[1, :2])
        ax_lyap = fig.add_subplot(gs[1, 2:])
        ax_dist = fig.add_subplot(gs[2, :2])
        ax_adj = fig.add_subplot(gs[2, 2:])

        time = np.arange(T)
        ax_states.plot(time, X.mean(axis=1), color='k', lw=2, label='mean')
        for i in selected:
            ax_states.plot(time, X[:,i], alpha=0.95, label=f'i={i}')
        ax_states.set_title('Agent state trajectories (highlighted)')
        ax_states.set_xlabel('time'); ax_states.legend(fontsize='small'); ax_states.grid(True)

        im_rsi = ax_rsi_heat.imshow(RSI_ts.T, aspect='auto', origin='lower',
                                extent=[0, T, 0, N], cmap='viridis', vmin=0, vmax=1)
        ax_rsi_heat.set_title('RSI heatmap (time × agent)')
        ax_rsi_heat.set_xlabel('time'); ax_rsi_heat.set_ylabel('agent')
        cbar1 = fig.colorbar(im_rsi, ax=ax_rsi_heat, fraction=0.046); cbar1.set_label('RSI')

        im_edb = ax_entropy_heat.imshow(EDB_ts.T, aspect='auto', origin='lower',
                                extent=[0, T, 0, N], cmap='magma')
        ax_entropy_heat.set_title('EDB (entropy) heatmap (time × agent)')
        ax_entropy_heat.set_xlabel('time'); ax_entropy_heat.set_ylabel('agent')
        cbar2 = fig.colorbar(im_edb, ax=ax_entropy_heat, fraction=0.046); cbar2.set_label('EDB')

        ax_lyap.plot(time, V_ts, color='m')
        ax_lyap.axhline(gap, color='r', linestyle='--', label=f'RM gap={gap:.3f}')
        ax_lyap.set_title('Lyapunov candidate V(t)')
        ax_lyap.set_xlabel('time')
        ax_lyap.legend(); ax_lyap.grid(True)

        ax_dist.hist(X[-1,:], bins=30, alpha=0.7, label='final x')
        ax_dist.hist(RSI_ts[-1,:], bins=30, alpha=0.6, label='final RSI')
        ax_dist.set_title('Final distributions (state & RSI)'); ax_dist.legend(); ax_dist.grid(True)

        ax_adj.clear()
        if plot_adj_chk.value:
            if HAS_NX:
                G = nx.from_numpy_array(A)
                pos = nx.spring_layout(G, seed=2) if N <= 200 else None
                nx.draw(G, pos=pos, ax=ax_adj, node_size=30 if N>100 else 120, with_labels=False)
                ax_adj.set_title('Adjacency graph (networkx)')
            else:
                ax_adj.imshow(A, cmap='gray', interpolation='nearest', aspect='auto')
                ax_adj.set_title('Adjacency matrix (no networkx)')
        else:
            ax_adj.set_title('Adjacency plotting disabled'); ax_adj.axis('off')

        plt.tight_layout()
        plt.show()

    # store for quantum call
    global LAST_SIM
    LAST_SIM = {'A':A, 'W':W, 'X':X, 'RSI':RSI_ts, 'EDB':EDB_ts, 'V':V_ts, 'params':{'a':a,'b':b,'eps':eps,'gamma':gamma,'K_RSI':K_RSI,'K_EDB':K_EDB}}
    status_lbl.value = 'Done'

run_button.on_click(run_multi_agent)

# -------------------------
# Quantum simulation runner (agent-specific)
# -------------------------
def run_quantum(_):
    out_area.clear_output(wait=True)
    quantum_status.value = 'Running quantum...'
    try:
        sim = LAST_SIM
    except Exception:
        with out_area:
            print("Run a classical multi-agent sim first (press 'Run simulation').")
        quantum_status.value = 'No classical sim'
        return
    A = sim['A']; W = sim['W']; X = sim['X']; RSI = sim['RSI']; EDB = sim['EDB']
    params = sim['params']
    a = params['a']; b = params['b']; eps = params['eps']
    agent = int(quantum_agent.value)
    T = X.shape[0]

    # Map classical x to initial Bloch vector for the agent
    # We'll map x -> z coordinate; add small orthogonal components for dynamics
    z0 = np.tanh(X[1,agent])  # map to (-1,1)
    x0 = 0.2 * np.sign(X[1,agent])  # small x component proportional to sign
    y0 = 0.05 * np.sin(0.5 * X[1,agent])
    b0 = np.array([x0, y0, z0])
    rho = bloch_to_rho(b0)

    # storage
    rhos = np.zeros((T,2,2), dtype=complex)
    purities = np.zeros(T)
    vnent = np.zeros(T)
    blochs = np.zeros((T,3))

    rhos[0,:,:] = rho
    purities[0] = purity(rho)
    vnent[0] = von_neumann_entropy(rho)
    blochs[0,:] = rho_to_bloch(rho)

    # Quantum update recipe (toy): unitary rotation whose angle depends on local coupling magnitude,
    # plus depolarizing channel weighted by eps, plus corrections from RSI and EDB.
    for t in range(0, T-1):
        # classical-driven rotation strength: local neighborhood influence magnitude
        local_influence = np.linalg.norm(W[agent,:]) if W is not None else 0.0
        # compute a driving angle from the agent's classical state magnitude
        theta = 0.2 * (np.abs(X[t,agent]) + local_influence)
        rho = unitary_rotation(rho, theta, axis='y')
        # depolarize proportional to eps
        rho = depolarizing_channel(rho, min(0.5, eps*5))
        # RSI encourages purification: apply a small dephasing-reduction if RSI high
        rsi_val = float(RSI[t,agent]) if t < RSI.shape[0] else 0.0
        edb_val = float(EDB[t,agent]) if t < EDB.shape[0] else 0.0
        # model: higher RSI -> apply imaginary purification channel (reduce dephasing)
        dephasing_strength = max(0.0, 0.2 - 0.4 * rsi_val)
        rho = dephasing_channel(rho, dephasing_strength)
        # model: higher EDB -> add depolarization (more mixed)
        rho = depolarizing_channel(rho, min(0.5, 0.4 * edb_val))
        # ensure hermitian + trace=1
        rho = (rho + rho.conj().T) / 2
        rho = rho / np.trace(rho)
        rhos[t+1,:,:] = rho
        purities[t+1] = purity(rho)
        vnent[t+1] = von_neumann_entropy(rho)
        blochs[t+1,:] = rho_to_bloch(rho)

    # Plot quantum + linked classical
    with out_area:
        fig = plt.figure(figsize=(14,10))
        gs = fig.add_gridspec(3,2, height_ratios=[1,1,1.1])
        ax1 = fig.add_subplot(gs[0,0])
        ax2 = fig.add_subplot(gs[0,1])
        ax3 = fig.add_subplot(gs[1,0])
        ax4 = fig.add_subplot(gs[1,1])
        ax5 = fig.add_subplot(gs[2,:])

        time = np.arange(T)
        # classical agent state trajectory
        ax1.plot(time, X[:,agent], label=f'classical x_{agent}')
        ax1.set_title(f'Classical state x_{agent}(t)'); ax1.grid(True)

        # purity and von neumann
        ax2.plot(time, purities, label='purity'); ax2.set_ylabel('purity'); ax2.legend(); ax2.grid(True)
        ax3.plot(time, vnent, label='von Neumann entropy'); ax3.set_ylabel('S(rho)'); ax3.legend(); ax3.grid(True)

        # Bloch components
        ax4.plot(time, blochs[:,0], label='x'); ax4.plot(time, blochs[:,1], label='y'); ax4.plot(time, blochs[:,2], label='z')
        ax4.set_title('Bloch vector components'); ax4.legend(); ax4.grid(True)

        # linked RSI vs purity (dual axis)
        ax5.plot(time, RSI[:,agent], label='RSI (classical)', color='C0')
        ax5.set_xlabel('time'); ax5.set_ylabel('RSI')
        ax5.grid(True)
        ax5b = ax5.twinx()
        ax5b.plot(time, purities, label='purity (quantum)', color='C1', alpha=0.8)
        ax5b.set_ylabel('purity')
        ax5.set_title('Classical RSI vs Quantum purity (linked)')

        plt.tight_layout()
        plt.show()
    quantum_status.value = 'Done'


run_quantum_button.on_click(run_quantum)

# Helpful note
with out_area:
    print('Dashboard live. Run "Run simulation" first, then select a quantum agent and press "Run quantum sim for selected agent".')
